In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset 

In [2]:
original = pd.read_parquet("/home/sm/AI/z_portfolio/models/Cyber/random_forest/UNSW_NB15_training-set.parquet")
original.to_csv("data.csv", index=False)
dataset = pd.read_csv('/home/sm/AI/z_portfolio/models/Cyber/random_forest/data.csv')
#dataset.head()



In [3]:
#print("Dataset Shape:", dataset.shape)  # Number of rows & columns
#print(dataset.info())  # Column names & data types
#print(dataset.describe())  # Summary statistics

# Check for Missing Values

In [4]:
print(dataset.isnull().sum().sum())

0


# Convert categorical features into numbers

In [5]:
from sklearn.preprocessing import LabelEncoder
categorical_cols = ["proto", "service", "state", "attack_cat"]

le = LabelEncoder()
for col in categorical_cols:
    dataset[col] = le.fit_transform(dataset[col])

In [6]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# Splitting the dataset into the Training set and Test set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [8]:
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

#  Training XGBoost on the Training set

In [18]:
from xgboost import XGBClassifier
#classifier = XGBClassifier()
classifier = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='logloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

# Predicting the Test set results

In [10]:
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix

In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[11246     0]
 [    0 23823]]


1.0

# Accuracy Score

In [12]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 1.0000


# Precision, Recall, and F1-Score

In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')

Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000


# ROC-AUC Score

In [14]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
print(f'ROC-AUC Score: {roc_auc:.4f}')

ROC-AUC Score: 1.0000


# Classification Report

In [15]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11246
           1       1.00      1.00      1.00     23823

    accuracy                           1.00     35069
   macro avg       1.00      1.00      1.00     35069
weighted avg       1.00      1.00      1.00     35069



# Applying k-Fold Cross Validation

In [16]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 100.00 %
Standard Deviation: 0.00 %


# Saving the model

In [19]:
import joblib
joblib.dump(classifier, "XGBoost.pkl")

['XGBoost.pkl']